# TaskBot

In [14]:
from typing import Optional, List, Union, Annotated

from datetime import date, datetime
from pydantic import BaseModel, Field

from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.runnables.config import RunnableConfig

from langchain_google_vertexai import ChatVertexAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

from promptgit import Prompt

from utils import object_to_xml

In [2]:
system = Prompt(
    prompt = 'You are an executive assistant to {user}. The tasks are {tasks}', variables=['user', 'tasks']
)

In [3]:
class Task(BaseModel):
    id: Optional[int] = None
    due: date
    description: str
    status: int = 0

class TaskList(BaseModel):
    last_update: datetime = Field(default_factory=datetime.now)
    tasks: List[Task] = Field(default = [])

    def __str__(self):
        return object_to_xml(self.model_dump()['tasks'], 'task')
    
    def add(self, due: date, description: str, status: int = 0):
        if len(self.tasks) > 0:
            id = max([t.id for t in self.tasks]) + 1
        else:
            id = 1
        self.tasks.append(Task(id = id, due = due, description = description, status = status))

In [4]:
l = TaskList()
l.add(date(2024, 10, 22), 'submit chemistry homework')
l.add(date(2024, 10, 23), 'second chapter in chemistry texbook')

In [6]:
class TaskState(BaseModel):
    user_id: int
    tasks: TaskList = None
    messages: Annotated[list, add_messages] = []

In [8]:
llm = ChatVertexAI(model='gemini-1.5-flash-002')

def initiate(state: TaskState, config: RunnableConfig):
    print(config)
    return {'messages': [
        SystemMessage(str(system).format(tasks=str(tasks[state.user_id]) ,**memory[state.user_id])), 
        HumanMessage('What are my tasks for today/')]
           }

def chatbot(state: TaskState):
    return {"messages": [llm.invoke(state.messages)]}

In [9]:
graph_builder = StateGraph(TaskState)

graph_builder.add_node("initiate", initiate)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "initiate")
graph_builder.add_edge("initiate", "chatbot")
graph_builder.add_edge("chatbot", END)

In [10]:
checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer, interrupt_before=['chatbot'])

In [11]:
config = {"configurable": {"thread_id": "42"}}

In [12]:
graph.invoke({'user_id': 42}, config)

{'metadata': {'thread_id': '1', 'langgraph_step': 1, 'langgraph_node': 'initiate', 'langgraph_triggers': ['start:initiate'], 'langgraph_path': ('__pregel_pull', 'initiate'), 'langgraph_checkpoint_ns': 'initiate:04be0524-cea1-759e-f40a-ab30bfc36b75'}, 'configurable': {'thread_id': '1', '__pregel_resuming': False, '__pregel_task_id': '04be0524-cea1-759e-f40a-ab30bfc36b75', '__pregel_send': functools.partial(<function local_write at 0x75ab13741e40>, <built-in method extend of collections.deque object at 0x75ab13579c60>, dict_keys(['__start__', 'initiate', 'chatbot'])), '__pregel_read': functools.partial(<function local_read at 0x75ab13741da0>, 1, {'v': 1, 'ts': '2024-10-24T20:33:12.219616+00:00', 'id': '1ef92473-048f-6816-8000-60d654740992', 'channel_values': {'user_id': 42, 'start:initiate': '__start__'}, 'channel_versions': {'__start__': '00000000000000000000000000000002.0.9107407969680784', 'user_id': '00000000000000000000000000000002.0.646590357344696', 'start:initiate': '000000000000

{'user_id': 42,
 'messages': [SystemMessage(content='You are an executive assistant to Boris Tvaroska. The tasks are <task><item><id>1</id><due>2024-10-22</due><description>submit chemistry homework</description><status>0</status></item><item><id>2</id><due>2024-10-23</due><description>second chapter in chemistry texbook</description><status>0</status></item></task>', additional_kwargs={}, response_metadata={}, id='14216210-f8e9-4430-ad56-e2de7362fd66'),
  HumanMessage(content='What are my tasks for today/', additional_kwargs={}, response_metadata={}, id='c0d5bb79-4416-4c92-bb77-b389f4672205')]}

In [13]:
graph.get_state(config)

StateSnapshot(values={'user_id': 42, 'messages': [SystemMessage(content='You are an executive assistant to Boris Tvaroska. The tasks are <task><item><id>1</id><due>2024-10-22</due><description>submit chemistry homework</description><status>0</status></item><item><id>2</id><due>2024-10-23</due><description>second chapter in chemistry texbook</description><status>0</status></item></task>', additional_kwargs={}, response_metadata={}, id='14216210-f8e9-4430-ad56-e2de7362fd66'), HumanMessage(content='What are my tasks for today/', additional_kwargs={}, response_metadata={}, id='c0d5bb79-4416-4c92-bb77-b389f4672205')]}, next=('chatbot',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1ef92473-0499-61cf-8001-d79ea2e95a63'}}, metadata={'source': 'loop', 'writes': {'initiate': {'messages': [SystemMessage(content='You are an executive assistant to Boris Tvaroska. The tasks are <task><item><id>1</id><due>2024-10-22</due><description>submit chemistry homework</d